In [48]:
import pandas as pd
import sys
from pyspark.sql import SparkSession, Row

In [54]:
import os
os.environ['SPARK_VERSION'] = '3.3'

In [47]:
filepath = sys.path[0]
filename = "products.csv"

In [64]:
file_df = pd.read_csv(f"{filepath}/{filename}")

In [65]:
file_df.head

<bound method NDFrame.head of        product_id   product_name  price
0               0      product_0     22
1               1      product_1     35
2               2      product_2    146
3               3      product_3     17
4               4      product_4     66
...           ...            ...    ...
99995       99995  product_99995     98
99996       99996  product_99996     66
99997       99997  product_99997    108
99998       99998  product_99998    131
99999       99999  product_99999    143

[100000 rows x 3 columns]>

In [81]:
import pydeequ
#Set up a PySpark session
spark = (SparkSession
    .builder
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())

# Convert pandas DataFrame to PySpark DataFrame
df = spark.createDataFrame(file_df)


df.show()

+----------+------------+-----+
|product_id|product_name|price|
+----------+------------+-----+
|         0|   product_0|   22|
|         1|   product_1|   35|
|         2|   product_2|  146|
|         3|   product_3|   17|
|         4|   product_4|   66|
|         5|   product_5|   31|
|         6|   product_6|  127|
|         7|   product_7|  116|
|         8|   product_8|  121|
|         9|   product_9|   98|
|        10|  product_10|   54|
|        11|  product_11|   25|
|        12|  product_12|  125|
|        13|  product_13|    8|
|        14|  product_14|  100|
|        15|  product_15|  111|
|        16|  product_16|    1|
|        17|  product_17|  115|
|        18|  product_18|   69|
|        19|  product_19|   59|
+----------+------------+-----+
only showing top 20 rows



In [83]:
#Analyzers
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness("price")) \
                    .run()

analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

+-------+--------+------------+--------+
| entity|instance|        name|   value|
+-------+--------+------------+--------+
|Dataset|       *|        Size|100000.0|
| Column|   price|Completeness|     1.0|
+-------+--------+------------+--------+



In [89]:
#Profile
import numpy
from pydeequ.profiles import *

# Profile
result = ColumnProfilerRunner(spark) \
    .onData(spark_df) \
    .run()

for col, profile in result.profiles.items():
    print(profile)

Py4JJavaError: An error occurred while calling o647.run.
: com.amazon.deequ.analyzers.runners.MetricCalculationRuntimeException: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 17.0 failed 1 times, most recent failure: Lost task 3.0 in stage 17.0 (TID 68) (f2e70ea9fc71 executor driver): java.lang.NoSuchMethodError: 'org.apache.spark.sql.types.StructType org.apache.spark.sql.types.StructType$.fromAttributes(scala.collection.Seq)'
	at org.apache.spark.sql.catalyst.expressions.aggregate.StatefulHyperloglogPlus.aggBufferSchema(StatefulHyperloglogPlus.scala:62)
	at org.apache.spark.sql.execution.aggregate.AggregationIterator.$anonfun$initializeAggregateFunctions$1(AggregationIterator.scala:110)
	at org.apache.spark.sql.execution.aggregate.AggregationIterator.$anonfun$initializeAggregateFunctions$1$adapted(AggregationIterator.scala:81)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.sql.execution.aggregate.AggregationIterato

In [90]:
#Constraint Suggestion

from pydeequ.suggestions import *

suggestionResult = ConstraintSuggestionRunner(spark) \
             .onData(df) \
             .addConstraintRule(DEFAULT()) \
             .run()

# Constraint Suggestions in JSON format
print(suggestionResult)


Py4JJavaError: An error occurred while calling o651.run.
: com.amazon.deequ.analyzers.runners.MetricCalculationRuntimeException: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 18.0 failed 1 times, most recent failure: Lost task 1.0 in stage 18.0 (TID 74) (f2e70ea9fc71 executor driver): java.lang.NoSuchMethodError: 'org.apache.spark.sql.types.StructType org.apache.spark.sql.types.StructType$.fromAttributes(scala.collection.Seq)'
	at org.apache.spark.sql.catalyst.expressions.aggregate.StatefulHyperloglogPlus.aggBufferSchema(StatefulHyperloglogPlus.scala:62)
	at org.apache.spark.sql.execution.aggregate.AggregationIterator.$anonfun$initializeAggregateFunctions$1(AggregationIterator.scala:110)
	at org.apache.spark.sql.execution.aggregate.AggregationIterator.$anonfun$initializeAggregateFunctions$1$adapted(AggregationIterator.scala:81)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.sql.execution.aggregate.AggregationIterato

In [97]:
#Constraint Verification

from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Warning, "Review Check")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.hasSize(lambda x: x >= 3) \
        .hasMin("price", lambda x: x == 0) \
        .isComplete("product_name")  \
        .isUnique("product_id")  \
        #.isContainedIn("product_id", [1,2,4,5,6,7]) \
        .isNonNegative("price")) \
    .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

# Convert the check results to a pandas DataFrame
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)

# Save the pandas DataFrame as an HTML file
checkResult_df.to_html("verification_report.html", index=False)

# Display the path to the saved HTML report
print("HTML report saved to: verification_report.html")

+------------+-----------+------------+--------------------+-----------------+--------------------+
|       check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+------------+-----------+------------+--------------------+-----------------+--------------------+
|Review Check|    Warning|     Warning|SizeConstraint(Si...|          Success|                    |
|Review Check|    Warning|     Warning|MinimumConstraint...|          Failure|Value: 1.0 does n...|
|Review Check|    Warning|     Warning|CompletenessConst...|          Success|                    |
|Review Check|    Warning|     Warning|UniquenessConstra...|          Success|                    |
|Review Check|    Warning|     Warning|ComplianceConstra...|          Success|                    |
+------------+-----------+------------+--------------------+-----------------+--------------------+



AttributeError: 'DataFrame' object has no attribute 'to_html'

In [94]:
#Repository

from pydeequ.repository import *
from pydeequ.analyzers import *

metrics_file = FileSystemMetricsRepository.helper_metrics_file(spark, 'metrics.json')
repository = FileSystemMetricsRepository(spark, metrics_file)
key_tags = {'tag': 'pydeequ hello world'}
resultKey = ResultKey(spark, ResultKey.current_milli_time(), key_tags)

analysisResult = AnalysisRunner(spark) \
    .onData(df) \
    .addAnalyzer(ApproxCountDistinct('product_id')) \
    .useRepository(repository) \
    .saveOrAppendResult(resultKey) \
    .run()

In [70]:
#To load previous runs, use the repository object to load previous results back in.

result_metrep_df = repository.load() \
    .before(ResultKey.current_milli_time()) \
    .forAnalyzers([ApproxCountDistinct('b')]) \
    .getSuccessMetricsAsDataFrame()

In [71]:
#Wrapping up

spark.sparkContext._gateway.shutdown_callback_server()
spark.stop()